# Ipython widgets experiments

In [52]:
# !pip install ipywidgets

In [82]:
# !pip install voila

In [53]:
!jupyter labextension list
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

JupyterLab v2.2.9
Known labextensions:
   app dir: /Users/Sourabh/.pyenv/versions/3.8.6/envs/aai/share/jupyter/lab
        @jupyter-widgets/jupyterlab-manager v2.0.0  enabled  OK


In [1]:
import ipywidgets as widgets

In [2]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

In [3]:
out = widgets.Output(layout={'border': '1px solid lightgrey'})
out

Output(layout=Layout(border='1px solid lightgrey'))

In [4]:
with out:
    for i in range(10):
        print(i, 'Hello world!')

In [5]:
with out:
    display(widgets.IntSlider())

In [16]:
out.clear_output()

In [7]:
@out.capture()
def function_with_captured_output():
    print('This goes into the output widget')
    raise Exception('As does this')

function_with_captured_output()

In [8]:
debug_view = widgets.Output(layout={'border': '1px solid black'})

@debug_view.capture(clear_output=True)    # sets the output panel
def bad_callback(event):
    print('This is about to explode')
    return 1.0 / 0.0

button = widgets.Button(
    description='click me to raise an exception',
    layout={'width': '300px'}
)
button.on_click(bad_callback)
button

Button(description='click me to raise an exception', layout=Layout(width='300px'), style=ButtonStyle())

In [9]:
debug_view

Output(layout=Layout(border='1px solid black'))

In [10]:
import ipywidgets as widgets
import logging

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%',
            'height': '160px',
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout',
            'output_type': 'stream',
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs

    def show_logs(self):
        """ Show the logs """
        display(self.out)

    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()


logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [11]:
handler.show_logs()

Output(layout=Layout(border='1px solid black', height='160px', width='100%'))

In [12]:
handler.clear_logs()
logger.info('Starting program')

try:
    logger.info('About to try something dangerous...')
    1.0/0.0
except Exception as e:
    logger.exception('An error occurred!')

In [13]:
import threading
import time
import itertools

def run():
    for i in itertools.count(0):
        time.sleep(1)
        print('output from background {}'.format(i))
        if i >= 10:
            break

t = threading.Thread(target=run)
t.start()

In [14]:
import threading
from IPython.display import display, HTML
import ipywidgets as widgets
import time

def thread_func(something, out):
    for i in range(1, 5):
        time.sleep(0.3)
        out.append_stdout('{} {} {}\n'.format(i, '**'*i, something))
    out.append_display_data(HTML("<em>All done!</em>"))

display('Display in main thread')
out = widgets.Output()
# Now the key: the container is displayed (while empty) in the main thread
display(out)

thread = threading.Thread(
    target=thread_func,
    args=("some text", out))
thread.start()

'Display in main thread'

Output()

In [15]:
thread.join()

output from background 0


In [28]:
plotout = widgets.Output()

In [39]:
display(plotout)

Output()

In [40]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
x = np.arange(start=1, stop=11, step=1)
y = np.random.randint(low=10, high=100, size=10, dtype=int)
x, y

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([55, 25, 44, 70, 84, 54, 77, 91, 88, 60]))

In [42]:
with plotout:
    plt.scatter(x, y)
    plt.show()

In [38]:
plotout.clear_output()

In [44]:
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [81]:
@widgets.interact_manual(
    lamda=(0.0, 20.0, 0.1),
    samples=(1000, 100000, 1000)
)
def plot_poisson_distribution_samples(lamda=15.0, samples=50000):
    dist_name = 'Poisson'
    sample_space = np.random.poisson(lamda, size=samples)
    
    # create subplot axes
    sns.set_style('white')
    n_r = 1
    n_c = 1
    fig, axis = plt.subplots(n_r, n_c, figsize=(7*n_c, 7*n_r))

    # sample mean as the statistic for sampling distribution
    plt_title = '\n'.join([f'{dist_name} Distribution', 'sample size comparison'])
    plt.suptitle(plt_title, y=0.98, verticalalignment='bottom', fontsize=20)
    sns.histplot(
        sample_space,
        bins=30,
        stat='density',
        #kde=True,
        color='orange',
        edgecolor='white',
        alpha=0.3,
        ax=axis,
        label=f'sample space size = {samples}',
        legend=False
    )
    print(lamda, type(lamda))
    print(samples, type(samples))
    print(type(sample_space), type(sample_space[0]))
    sns.kdeplot(
        sample_space,
        color='blueviolet',
        lw=2,
        ax=axis
    )
    #axis.legend(loc='best', prop={'size': 12})
    axis.set_title(f'Sample space size = {samples}')
    axis.title.set_size(18)
    axis.set_xlabel('Bins', fontsize=14)
    axis.yaxis.label.set_size(14)


interactive(children=(FloatSlider(value=15.0, description='lamda', max=20.0), IntSlider(value=50000, descripti…

---